# Project 4 - Data Cleaning
---

In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv("./input/train.csv")
df_test = pd.read_csv("./input/test.csv")
df_spray = pd.read_csv("./input/spray.csv")
df_weather = pd.read_csv("./input/weather.csv")

### Data Cleaning for weather

In [3]:
df_weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


### Feature selection

Factors affecting mosquito breeding are temperature, humidity and precipitation - mosquitos thrive in hot and humid conditions, less so in cold and dry conditions. Hence, the said weather features are used in our analysis.

In [4]:
weather_v1 = df_weather
weather_v1 = weather_v1[
    ['Station',
     'Date',
     'Tmax',
     'Tmin',
     'Tavg',
     'DewPoint',
     'WetBulb',
     'PrecipTotal']
]

# Handle Missing and Trace values.
# Change data type to float for numerical values.
weather_v1.replace('M', np.nan, inplace=True)
weather_v1.replace('  T', 0.0, inplace=True)
weather_v1.PrecipTotal = weather_v1.PrecipTotal.astype(float)
weather_v1.Tavg = weather_v1.Tavg.astype(float)
weather_v1.WetBulb = weather_v1.WetBulb.astype(float)

In [5]:
weather_v1.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal
0,1,2007-05-01,83,50,67.0,51,56.0,0.0
1,2,2007-05-01,84,52,68.0,51,57.0,0.0
2,1,2007-05-02,59,42,51.0,42,47.0,0.0
3,2,2007-05-02,60,43,52.0,42,47.0,0.0
4,1,2007-05-03,66,46,56.0,40,48.0,0.0


In [6]:
# Group weather data by their station
weather_stn_1 = weather_v1.groupby('Station').get_group(1)
weather_stn_1.drop('Station', axis=1, inplace=True)
weather_stn_2= weather_v1.groupby('Station').get_group(2)
weather_stn_2.drop('Station', axis=1, inplace=True)

# Fill missing values in weather data with previous day's value using fill forward method
weather_stn_1 = weather_stn_1.fillna(method='ffill')
weather_stn_2 = weather_stn_2.fillna(method='ffill')

# final check 
print('null values in weather station 1\n', weather_stn_1.isnull().sum())
print('--------------------------------')
print('null values in weather station 2\n', weather_stn_2.isnull().sum())

null values in weather station 1
 Date           0
Tmax           0
Tmin           0
Tavg           0
DewPoint       0
WetBulb        0
PrecipTotal    0
dtype: int64
--------------------------------
null values in weather station 2
 Date           0
Tmax           0
Tmin           0
Tavg           0
DewPoint       0
WetBulb        0
PrecipTotal    0
dtype: int64


In [7]:
# reset index
weather_stn_1.reset_index(drop=True, inplace=True)
weather_stn_2.reset_index(drop=True, inplace=True)

In [8]:
print(weather_stn_1.shape)
weather_stn_1.head()

(1472, 7)


,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal
0,2007-05-01,83,50,67.0,51,56.0,0.0
1,2007-05-02,59,42,51.0,42,47.0,0.0
2,2007-05-03,66,46,56.0,40,48.0,0.0
3,2007-05-04,66,49,58.0,41,50.0,0.0
4,2007-05-05,66,53,60.0,38,49.0,0.0


In [9]:
print(weather_stn_2.shape)
weather_stn_2.head()

(1472, 7)


,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal
0,2007-05-01,84,52,68.0,51,57.0,0.0
1,2007-05-02,60,43,52.0,42,47.0,0.0
2,2007-05-03,67,48,58.0,40,50.0,0.0
3,2007-05-04,78,51,58.0,42,50.0,0.0
4,2007-05-05,66,54,60.0,39,50.0,0.0


### Feature selection - 2nd round
To reduce complexity, values from both weather stations are averaged.

In [10]:
# Taking the average temperature from both station 1 and 2
weather_stn_avg = pd.DataFrame()
weather_stn_avg['Date'] = weather_stn_1['Date']
for col in ['Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb', 'PrecipTotal']:
    weather_stn_avg[col] = (weather_stn_1[col] + weather_stn_2[col])*.5

In [11]:
weather_stn_avg.head()

,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal
0,2007-05-01,83.5,51.0,67.5,51.0,56.5,0.0
1,2007-05-02,59.5,42.5,51.5,42.0,47.0,0.0
2,2007-05-03,66.5,47.0,57.0,40.0,49.0,0.0
3,2007-05-04,72.0,50.0,58.0,41.5,50.0,0.0
4,2007-05-05,66.0,53.5,60.0,38.5,49.5,0.0


In [12]:
# export to CSV file 

weather_stn_avg.to_csv('./input/weather_stn_avg.csv')

---

### Data Cleaning for train dataset

In [13]:
print(f"Training set has {df_train.shape[0]} rows and {df_train.shape[1]} columns.")
df_train.head(1)

Training set has 10506 rows and 12 columns.


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [14]:
print(f"Test set has {df_test.shape[0]} rows and {df_test.shape[1]} columns.")
df_test.head(1)

Test set has 116293 rows and 11 columns.


,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [15]:
#Combine both datasets for pre-processing.
df_train['source'] = 1 # To identify which is train and test data. 1 for train.
df_test['source'] = 0

data = pd.concat([df_train,df_test], ignore_index = True, sort = False)

print(f"Combined training and test set has {data.shape[0]} rows and {data.shape[1]} columns.")
data.head(3)

Combined training and test set has 126799 rows and 14 columns.


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,source,Id
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1.0,0.0,1,NaN
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1.0,0.0,1,NaN
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1.0,0.0,1,NaN


In [16]:
data['source'].value_counts()

0    116293
1     10506
Name: source, dtype: int64

In [17]:
print(data.isnull().sum().sort_values(ascending=False).head(30))

WnvPresent                116293
NumMosquitos              116293
Id                         10506
source                         0
AddressAccuracy                0
Longitude                      0
Latitude                       0
AddressNumberAndStreet         0
Trap                           0
Street                         0
Block                          0
Species                        0
Address                        0
Date                           0
dtype: int64


In [18]:
data.columns.tolist()

['Date',
 'Address',
 'Species',
 'Block',
 'Street',
 'Trap',
 'AddressNumberAndStreet',
 'Latitude',
 'Longitude',
 'AddressAccuracy',
 'NumMosquitos',
 'WnvPresent',
 'source',
 'Id']

Drop _NumMosquitos_ since test set does not have this feature.

In [19]:
data.drop(['Address',
           'Block',
           'Street',
           'AddressNumberAndStreet',
           'AddressAccuracy',
           'NumMosquitos',
           'Id'], axis=1, inplace=True)

In [20]:
data.head(1)

,Date,Species,Trap,Latitude,Longitude,WnvPresent,source
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.95469,-87.800991,0.0,1


In [21]:
data = pd.merge(data,weather_stn_avg, on='Date', how='inner')

In [22]:
data.head()

,Date,Species,Trap,Latitude,Longitude,WnvPresent,source,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0
3,2007-05-29,CULEX PIPIENS/RESTUANS,T015,41.974089,-87.824812,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0
4,2007-05-29,CULEX RESTUANS,T015,41.974089,-87.824812,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0


In [23]:
print(data.isnull().sum().sort_values(ascending=False).head(30))

WnvPresent     116293
PrecipTotal         0
WetBulb             0
DewPoint            0
Tavg                0
Tmin                0
Tmax                0
source              0
Longitude           0
Latitude            0
Trap                0
Species             0
Date                0
dtype: int64


### Date features

In [24]:
data['Date'] = pd.to_datetime(data['Date'])

In [25]:
data.head(3)

,Date,Species,Trap,Latitude,Longitude,WnvPresent,source,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126799 entries, 0 to 126798
Data columns (total 13 columns):
Date           126799 non-null datetime64[ns]
Species        126799 non-null object
Trap           126799 non-null object
Latitude       126799 non-null float64
Longitude      126799 non-null float64
WnvPresent     10506 non-null float64
source         126799 non-null int64
Tmax           126799 non-null float64
Tmin           126799 non-null float64
Tavg           126799 non-null float64
DewPoint       126799 non-null float64
WetBulb        126799 non-null float64
PrecipTotal    126799 non-null float64
dtypes: datetime64[ns](1), float64(9), int64(1), object(2)
memory usage: 13.5+ MB


In [27]:
data['Month'] = data['Date'].dt.month
data['Year'] = data['Date'].dt.year
data['Day'] = data['Date'].dt.day
data['Week'] = data['Date'].dt.week

In [28]:
data.head(3)

,Date,Species,Trap,Latitude,Longitude,WnvPresent,source,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal,Month,Year,Day,Week
0,2007-05-29,CULEX PIPIENS/RESTUANS,T002,41.954690,-87.800991,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0,5,2007,29,22
1,2007-05-29,CULEX RESTUANS,T002,41.954690,-87.800991,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0,5,2007,29,22
2,2007-05-29,CULEX RESTUANS,T007,41.994991,-87.769279,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0,5,2007,29,22


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126799 entries, 0 to 126798
Data columns (total 17 columns):
Date           126799 non-null datetime64[ns]
Species        126799 non-null object
Trap           126799 non-null object
Latitude       126799 non-null float64
Longitude      126799 non-null float64
WnvPresent     10506 non-null float64
source         126799 non-null int64
Tmax           126799 non-null float64
Tmin           126799 non-null float64
Tavg           126799 non-null float64
DewPoint       126799 non-null float64
WetBulb        126799 non-null float64
PrecipTotal    126799 non-null float64
Month          126799 non-null int64
Year           126799 non-null int64
Day            126799 non-null int64
Week           126799 non-null int64
dtypes: datetime64[ns](1), float64(9), int64(5), object(2)
memory usage: 17.4+ MB


In [30]:
# This creates a CSV that will be used for feature engineering.
data.to_csv('./output/data_v0.csv')

### Get dummies for species and traps features

This creates a CSV that will be used for modelling. This does not have any feature engineering.

In [31]:
data.drop(['Date'],axis=1,inplace=True)

In [32]:
data = pd.get_dummies(data=data)

In [33]:
data.shape

(126799, 171)

In [34]:
data.head(3)

,Latitude,Longitude,WnvPresent,source,Tmax,Tmin,Tavg,DewPoint,WetBulb,PrecipTotal,Month,Year,Day,Week,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,Species_UNSPECIFIED CULEX,Trap_T001,Trap_T002,Trap_T002A,Trap_T002B,Trap_T003,Trap_T004,Trap_T005,Trap_T006,Trap_T007,Trap_T008,Trap_T009,Trap_T011,Trap_T012,Trap_T013,Trap_T014,Trap_T015,Trap_T016,Trap_T017,Trap_T018,Trap_T019,Trap_T025,Trap_T027,Trap_T028,Trap_T030,Trap_T031,Trap_T033,Trap_T034,Trap_T035,Trap_T036,Trap_T037,Trap_T039,Trap_T040,Trap_T043,Trap_T044,Trap_T045,Trap_T046,Trap_T047,Trap_T048,Trap_T049,Trap_T050,Trap_T051,Trap_T054,Trap_T054C,Trap_T060,Trap_T061,Trap_T062,Trap_T063,Trap_T065,Trap_T065A,Trap_T066,Trap_T067,Trap_T069,Trap_T070,Trap_T071,Trap_T072,Trap_T073,Trap_T074,Trap_T075,Trap_T076,Trap_T077,Trap_T078,Trap_T079,Trap_T080,Trap_T081,Trap_T082,Trap_T083,Trap_T084,Trap_T085,Trap_T086,Trap_T088,Trap_T089,Trap_T090,Trap_T090A,Trap_T090B,Trap_T090C,Trap_T091,Trap_T092,Trap_T094,Trap_T094B,Trap_T095,Trap_T096,Trap_T097,Trap_T099,Trap_T100,Trap_T102,Trap_T103,Trap_T107,Trap_T114,Trap_T115,Trap_T128,Trap_T128A,Trap_T129,Trap_T135,Trap_T138,Trap_T141,Trap_T142,Trap_T143,Trap_T144,Trap_T145,Trap_T146,Trap_T147,Trap_T148,Trap_T149,Trap_T150,Trap_T151,Trap_T152,Trap_T153,Trap_T154,Trap_T155,Trap_T156,Trap_T157,Trap_T158,Trap_T159,Trap_T160,Trap_T161,Trap_T162,Trap_T200,Trap_T200A,Trap_T200B,Trap_T206,Trap_T209,Trap_T212,Trap_T215,Trap_T218,Trap_T218A,Trap_T218B,Trap_T218C,Trap_T219,Trap_T220,Trap_T221,Trap_T222,Trap_T223,Trap_T224,Trap_T225,Trap_T226,Trap_T227,Trap_T228,Trap_T229,Trap_T230,Trap_T231,Trap_T232,Trap_T233,Trap_T234,Trap_T235,Trap_T236,Trap_T237,Trap_T238,Trap_T900,Trap_T903
0,41.954690,-87.800991,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0,5,2007,29,22,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,41.954690,-87.800991,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0,5,2007,29,22,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,41.994991,-87.769279,0.0,1,88.0,62.5,75.5,58.5,65.5,0.0,5,2007,29,22,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
# This creates a CSV that will be used for modelling. This does not have any feature engineering.
data.to_csv('./output/data_nofeature.csv')